###### Imports and Pulls

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

###### API Key

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

###### Geography Lists

In [3]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson

In [4]:
slaces = ['02180', #Ashland City: Cheatham
          '39660', #Kingston Springs: Cheatham
          '57480', #Pegram: Cheatham
          '59560', #Pleasant View: Cheatham
          '04620', #Belle Meade: Davidson
          '05140', #Berry Hill: Davidson
          '27020', #Forest Hills: Davidson
          '29920', #Goodlettsville: Davidson/Sumner
          '52006', #Nashville-Davidson metropolitan government (balance): Davidson
          '54780', #Oak Hill: Davidson
          '63140', #Ridgetop: Davidson/Robertson
          '09880', #Burns: Dickson
          '13080', #Charlotte: Dickson
          '20620', #Dickson: Dickson
          '69080', #Slayden: Dickson
          '76860', #Vanleer: Dickson
          '79980', #White Bluff: Dickson
          '24320', #Erin: Houston
          '73460', #Tennessee Ridge: Houston/Stewart
          '44840', #McEwen: Humphreys
          '52820', #New Johnsonville: Humphreys
          '78560', #Waverly: Humphreys
          '16540', #Columbia: Maury
          '51080', #Mount Pleasant: Maury
          '70580', #Spring Hill: Maury/Williamson
          '15160', #Clarksville: Montgomery
          '00200', #Adams: Robertson
          '11980', #Cedar Hill: Robertson
          '16980', #Coopertown: Robertson
          '18420', #Cross Plains: Robertson
          '30960', #Greenbrier: Robertson
          '48980', #Millersville: Robertson/Sumner
          '60280', #Portland: Robertson/Sumner
          '70500', #Springfield: Robertson
          '80200', #White House: Robertson/Sumner
          '22360', #Eagleville: Rutherford
          '41200', #La Vergne: Rutherford
          '51560', #Murfreesboro: Rutherford
          '69420', #Smyrna: Rutherford
          '18820', #Cumberland City: Stewart
          '21400', #Dover: Stewart
          '28540', #Gallatin: Sumner
          '33280', #Hendersonville: Sumner
          '79420', #Westmoreland: Sumner
          '08280', #Brentwood: Williamson
          '25440', #Fairview: Williamson
          '27740', #Franklin: Williamson
          '53460', #Nolensville: Williamson
          '73900', #Thompson's Station: Williamson
          '41520', #Lebanon: Wilson
          '50780', #Mount Juliet: Wilson
          '78320', #Watertown: Wilson
          '40160', #Lafayette: Macon
          '69114', #Scottsville city, Kentucky
          '28918', #Franklin city, Kentucky
          '49460'] #Mitchelville city: Sumner

At this time the only variable from the subject tables is "average commute time to work" so there is no data guide. In 2010 and 2020 the variable remains consistent: S0801_C01_046E

In [5]:
#2010 ACS 5 Year Subject Table Estimate
#counties
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [6]:
twentyten = savename

In [7]:
twentyten['Mean Travel Time to Work'] = twentyten['Mean Travel Time to Work'].astype(float)

In [8]:
twentyten.tail()

,NAME,GEO_ID,Mean Travel Time to Work,StateFIPS,GeoFIPS
413,"Westmoreland town, Tennessee",1600000US4779420,33.4,47,79420
414,"White Bluff town, Tennessee",1600000US4779980,32.6,47,79980
415,"White House city, Tennessee",1600000US4780200,28.8,47,80200
0,Tennessee,0400000US47,23.9,47,0
0,United States,0100000US,25.2,1,0


In [16]:
#2021 ACS 5 Year Subject Table Estimate
#counties
url_str= 'https://api.census.gov/data/2021/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [17]:
twentytwentyone = savename

In [18]:
twentytwentyone['Mean Travel Time to Work'] = twentytwentyone['Mean Travel Time to Work'].astype(float)

In [19]:
twentytwentyone.tail()

,NAME,GEO_ID,Mean Travel Time to Work,StateFIPS,GeoFIPS
181,"Fort Knox CDP, Kentucky",1600000US2128540,12.5,21,28540
189,"Franklin city, Kentucky",1600000US2128918,18.9,21,28918
451,"Scottsville city, Kentucky",1600000US2169114,21.8,21,69114
0,Tennessee,0400000US47,25.5,47,0
0,United States,0100000US,26.8,1,0


In [13]:
twentyten = twentyten.drop(columns = ['StateFIPS', 'GeoFIPS'])

In [14]:
twentyten = twentyten.set_index('NAME').transpose()
#rename geographies dict
twentyten = twentyten.rename(columns = {'Allen County, Kentucky': 'Allen County, KY', 'Cheatham County, Tennessee': 'Cheatham County', 
                              'Davidson County, Tennessee': 'Davidson County', 'Dickson County, Tennessee': 'Dickson County', 
                              'Houston County, Tennessee': 'Houston County', 'Humphreys County, Tennessee': 'Humphreys County', 
                              'Maury County, Tennessee': 'Maury County', 'Montgomery County, Tennessee': 'Montgomery County', 
                              'Robertson County, Tennessee': 'Robertson County', 'Rutherford County, Tennessee': 'Rutherford County', 
                              'Simpson County, Kentucky': 'Simpson County, KY', 'Stewart County, Tennessee': 'Stewart County', 
                              'Sumner County, Tennessee': 'Sumner County', 'Trousdale County, Tennessee': 'Trousdale County', 
                              'Williamson County, Tennessee': 'Williamson County', 'Wilson County, Tennessee': 'Wilson County', 
                              'Adams city, Tennessee': 'Adams', 'Ashland City town, Tennessee': 'Ashland City', 'Belle Meade city, Tennessee': 'Belle Meade', 
                              'Berry Hill city, Tennessee': 'Berry Hill', 'Brentwood city, Tennessee': 'Brentwood', 'Burns town, Tennessee': 'Burns', 
                              'Cedar Hill city, Tennessee': 'Cedar Hill', 'Charlotte town, Tennessee': 'Charlotte', 'Clarksville city, Tennessee': 'Clarksville', 
                              'Columbia city, Tennessee': 'Columbia', 'Coopertown town, Tennessee': 'Coopertown', 'Cross Plains city, Tennessee': 'Cross Plains', 
                              'Cumberland City town, Tennessee': 'Cumberland City', 'Dickson city, Tennessee': 'Dickson', 'Dover city, Tennessee': 'Dover', 
                              'Eagleville city, Tennessee': 'Eagleville', 'Erin city, Tennessee': 'Erin', 'Fairview city, Tennessee': 'Fairview', 
                              'Forest Hills city, Tennessee': 'Forest Hills', 'Franklin city, Tennessee': 'Franklin', 'Gallatin city, Tennessee': 'Gallatin', 
                              'Goodlettsville city, Tennessee': 'Goodlettsville', 'Greenbrier town, Tennessee': 'Greenbrier', 
                              'Hendersonville city, Tennessee': 'Hendersonville', 'Kingston Springs town, Tennessee': 'Kingston Springs', 
                              'La Vergne city, Tennessee': 'La Vergne', 'Lafayette city, Tennessee': 'Lafayette', 'Lebanon city, Tennessee': 'Lebanon', 
                              'McEwen city, Tennessee': 'McEwen', 'Millersville city, Tennessee': 'Millersville', 'Mitchellville city, Tennessee': 'Mitchellville', 
                              'Mount Juliet city, Tennessee': 'Mount Juliet', 'Mount Pleasant city, Tennessee': 'Mount Pleasant', 
                              'Murfreesboro city, Tennessee': 'Murfreesboro', 'Nashville-Davidson metropolitan government (balance)': 'Nashville', 
                              'New Johnsonville city, Tennessee': 'New Johnsonville', 'Nolensville town, Tennessee': 'Nolensville', 
                              'Oak Hill city, Tennessee': 'Oak Hill', 'Pegram town, Tennessee': 'Pegram', 'Pleasant View city, Tennessee': 'Pleasant View', 
                              'Portland city, Tennessee': 'Portland', 'Ridgetop city, Tennessee': 'Ridgetop', 'Slayden town, Tennessee': 'Slayden', 
                              'Smyrna town, Tennessee': 'Smyrna', 'Spring Hill city, Tennessee': 'Spring Hill', 'Springfield city, Tennessee': 'Springfield', 
                              'Tennessee Ridge town, Tennessee': 'Tennessee Ridge', "Thompson's Station town, Tennessee": "Thompson's Station", 
                              'Vanleer town, Tennessee': 'Vanleer', 'Watertown city, Tennessee': 'Watertown', 'Waverly city, Tennessee': 'Waverly', 
                              'Westmoreland town, Tennessee': 'Westmoreland', 'White Bluff town, Tennessee': 'White Bluff', 
                              'White House city, Tennessee': 'White House', 'Franklin city, Kentucky': 'Franklin, KY', 
                              'Scottsville city, Kentucky': 'Scottsville, KY', 'United States':'US'})
twentyten = twentyten.transpose().reset_index(drop = False)

In [15]:
twentyten.head(3)

,NAME,GEO_ID,Mean Travel Time to Work
0,Stewart County,0500000US47161,32.0
1,Sumner County,0500000US47165,27.6
2,Trousdale County,0500000US47169,31.9


In [16]:
twentyten.to_csv('../../Outputs/2010Subject.csv', index = False)

In [20]:
twentytwentyone = twentytwentyone.drop(columns = ['StateFIPS', 'GeoFIPS'])

In [21]:
twentytwentyone = twentytwentyone.set_index('NAME').transpose()
#rename geographies dict
twentytwentyone = twentytwentyone.rename(columns = {'Allen County, Kentucky': 'Allen County, KY', 'Cheatham County, Tennessee': 'Cheatham County', 
                              'Davidson County, Tennessee': 'Davidson County', 'Dickson County, Tennessee': 'Dickson County', 
                              'Houston County, Tennessee': 'Houston County', 'Humphreys County, Tennessee': 'Humphreys County', 
                              'Maury County, Tennessee': 'Maury County', 'Montgomery County, Tennessee': 'Montgomery County', 
                              'Robertson County, Tennessee': 'Robertson County', 'Rutherford County, Tennessee': 'Rutherford County', 
                              'Simpson County, Kentucky': 'Simpson County, KY', 'Stewart County, Tennessee': 'Stewart County', 
                              'Sumner County, Tennessee': 'Sumner County', 'Trousdale County, Tennessee': 'Trousdale County', 
                              'Williamson County, Tennessee': 'Williamson County', 'Wilson County, Tennessee': 'Wilson County', 
                              'Adams city, Tennessee': 'Adams', 'Ashland City town, Tennessee': 'Ashland City', 'Belle Meade city, Tennessee': 'Belle Meade', 
                              'Berry Hill city, Tennessee': 'Berry Hill', 'Brentwood city, Tennessee': 'Brentwood', 'Burns town, Tennessee': 'Burns', 
                              'Cedar Hill city, Tennessee': 'Cedar Hill', 'Charlotte town, Tennessee': 'Charlotte', 'Clarksville city, Tennessee': 'Clarksville', 
                              'Columbia city, Tennessee': 'Columbia', 'Coopertown town, Tennessee': 'Coopertown', 'Cross Plains city, Tennessee': 'Cross Plains', 
                              'Cumberland City town, Tennessee': 'Cumberland City', 'Dickson city, Tennessee': 'Dickson', 'Dover city, Tennessee': 'Dover', 
                              'Eagleville city, Tennessee': 'Eagleville', 'Erin city, Tennessee': 'Erin', 'Fairview city, Tennessee': 'Fairview', 
                              'Forest Hills city, Tennessee': 'Forest Hills', 'Franklin city, Tennessee': 'Franklin', 'Gallatin city, Tennessee': 'Gallatin', 
                              'Goodlettsville city, Tennessee': 'Goodlettsville', 'Greenbrier town, Tennessee': 'Greenbrier', 
                              'Hendersonville city, Tennessee': 'Hendersonville', 'Kingston Springs town, Tennessee': 'Kingston Springs', 
                              'La Vergne city, Tennessee': 'La Vergne', 'Lafayette city, Tennessee': 'Lafayette', 'Lebanon city, Tennessee': 'Lebanon', 
                              'McEwen city, Tennessee': 'McEwen', 'Millersville city, Tennessee': 'Millersville', 'Mitchellville city, Tennessee': 'Mitchellville', 
                              'Mount Juliet city, Tennessee': 'Mount Juliet', 'Mount Pleasant city, Tennessee': 'Mount Pleasant', 
                              'Murfreesboro city, Tennessee': 'Murfreesboro', 'Nashville-Davidson metropolitan government (balance)': 'Nashville', 
                              'New Johnsonville city, Tennessee': 'New Johnsonville', 'Nolensville town, Tennessee': 'Nolensville', 
                              'Oak Hill city, Tennessee': 'Oak Hill', 'Pegram town, Tennessee': 'Pegram', 'Pleasant View city, Tennessee': 'Pleasant View', 
                              'Portland city, Tennessee': 'Portland', 'Ridgetop city, Tennessee': 'Ridgetop', 'Slayden town, Tennessee': 'Slayden', 
                              'Smyrna town, Tennessee': 'Smyrna', 'Spring Hill city, Tennessee': 'Spring Hill', 'Springfield city, Tennessee': 'Springfield', 
                              'Tennessee Ridge town, Tennessee': 'Tennessee Ridge', "Thompson's Station town, Tennessee": "Thompson's Station", 
                              'Vanleer town, Tennessee': 'Vanleer', 'Watertown city, Tennessee': 'Watertown', 'Waverly city, Tennessee': 'Waverly', 
                              'Westmoreland town, Tennessee': 'Westmoreland', 'White Bluff town, Tennessee': 'White Bluff', 
                              'White House city, Tennessee': 'White House', 'Franklin city, Kentucky': 'Franklin, KY', 
                              'Scottsville city, Kentucky': 'Scottsville, KY', 'United States': 'US'})
twentytwentyone = twentytwentyone.transpose().reset_index(drop = False)

In [22]:
twentytwentyone.to_csv('../../Outputs/2021Subject.csv', index = False)